# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [57]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [58]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")

# Import Data

In [59]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Downloads/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Downloads/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [83]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    housing_per_year = sfo_data.groupby(by=[sfo_data.index])["housing_units"].mean()
    housing_per_year = sfo_data.reset_index()
    housing_per_year.head()    
    
    return px.bar(housing_per_year,x="year",y="housing_units",title='Housing Units in SF Per Year')


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    
    sf_grossrent = sfo_data.groupby(by=[sfo_data.index])["gross_rent"].mean()
    
    return px.line(sf_grossrent, x="year", y="gross_rent", title='Avg Gross Rent SF')



def average_sales_price():
    """Average Sales Price Per Year."""
    sf_ppsqft = sfo_data.groupby(by=[sfo_data.index])["sale_price_sqr_foot"].mean()
    
    return px.line(sf_ppsqft, x="year", y="sale_price_sqr_foot", title='Avg Sale Price Per Sqft')


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    avg_Pr_Neighborhood_df = sfo_data.groupby([sfo_data.index,'neighborhood']).mean()
    avg_Pr_Neighborhood_df.reset_index(inplace=True)
    avg_Pr_Neighborhood_df
    avg_Pr_Neighborhood_df.drop(['housing_units', 'gross_rent'], axis=1)
    
    return avg_Pr_Neighborhood_df.hvplot.line(x='year',y='sale_price_sqr_foot',groupby='neighborhood')

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    expensive_neighborhood = sfo_data.groupby(['neighborhood']).mean()
    expensive_neighborhood = expensive_neighborhood.sort_values(by='sale_price_sqr_foot')
    expensive_neighborhood = expensive_neighborhood.head(10)
    
    return expensive_neighborhood.hvplot.bar(x="neighborhood", y="sale_price_sqr_foot",title='Most Expensive Neighborhoods', rot=90)

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    expensive_neighborhood = sfo_data.groupby(['neighborhood']).mean()
    expensive_neighborhood = expensive_neighborhood.sort_values(by='sale_price_sqr_foot')
    expensive_neighborhood = expensive_neighborhood.head(10)
    
    return px.parallel_coordinates(expensive_neighborhood, dimensions=["sale_price_sqr_foot", "housing_units", "gross_rent"], color="sale_price_sqr_foot")


def parallel_categories():
    """Parallel Categories Plot."""
    expensive_neighborhood = sfo_data.groupby(['neighborhood']).mean()
    expensive_neighborhood = expensive_neighborhood.sort_values(by='sale_price_sqr_foot')
    expensive_neighborhood = expensive_neighborhood.head(10)
    
    return px.parallel_categories(expensive_neighborhood,color="sale_price_sqr_foot", color_continuous_scale=px.colors.cyclical.IceFire)


def neighborhood_map():
    """Neighborhood Map"""
    
    df_neighborhood_locations = pd.read_csv(file_path)
    df_neighborhood_locations.head()
    mean_neighbor = sfo_data.groupby(['neighborhood']).mean()
    mean_neighbor = mean_neighbor.sort_values(by='neighborhood')
    mean_neighbor.reset_index(inplace=True)
    mean_values_joined = pd.concat([mean_neighbor, df_neighborhood_locations],axis=1)
    mean_values_joined = mean_values_joined.drop(columns='Neighborhood')
    
    return px.scatter_mapbox(mean_values_joined, lat="Lat", lon="Lon", color="sale_price_sqr_foot", zoom=10, mapbox_style='open-street-map')




## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [86]:
start_here = pn.Column('Housing in SF is very expensive. Its best to find a good neighborhood.', neighborhood_map,top_most_expensive_neighborhoods,average_price_by_neighborhood)
parallel_plots = pn.Column('IF you can find a house at a lower price, you might be able to rent it for a lot', parallel_coordinates, parallel_categories)
available_units = pn.Column('Although, according to this, you might not have a place to live at all if youre not careful!', housing_units_per_year)

panel = pn.Tabs(
    ('Start Here',start_here),
    ('Buy to Rent?',parallel_plots),
    ('..Or just find a house!', neighborhood_plots)
    
)

## Serve the Panel Dashboard

In [87]:
panel.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive04049')
        [2] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive04055')
        [3] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive04140')
                    [0] HoloViews(DynamicMap, name='interactive04140')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
    [1] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive04181')
        [2] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive04187')
    [2] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive03579')